In [ ]:
import sys
import os
import subprocess
import shutil
from pathlib import Path
import time

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def run_command(cmd, check=True, shell=False):
    """Helper function to run commands and handle errors"""
    try:
        print(f"Executando comando: {' '.join(str(x) for x in cmd)}")
        result = subprocess.run(cmd, check=check, shell=shell, 
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
        print(f"Output: {result.stdout}")
        return result
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {' '.join(str(x) for x in cmd)}")
        print(f"Error output: {e.stderr}")
        return None

def clone_repository():
    """Clone the GitHub repository if in Colab"""
    if IN_COLAB:
        # Change to /content directory
        os.chdir('/content')
        
        # Remove existing directory if it exists
        if os.path.exists('/content/Anomaly_Detect_LSTM_VAE'):
            print("Removing existing repository...")
            shutil.rmtree('/content/Anomaly_Detect_LSTM_VAE')
        
        print("Cloning repository...")
        # Clone the repository
        cmd = ["git", "clone", "https://github.com/paya54/Anomaly_Detect_LSTM_VAE.git"]
        result = run_command(cmd)
        
        if result is None or result.returncode != 0:
            print("Failed to clone repository")
            return False
            
        # Verify the repository was cloned
        if not os.path.exists('/content/Anomaly_Detect_LSTM_VAE'):
            print("Repository directory not found after cloning")
            return False
            
        print("Repository cloned successfully")
        # Change to the repository directory
        os.chdir('/content/Anomaly_Detect_LSTM_VAE')
        return True
    return True

def get_project_dir():
    """Get the project directory in Google Colab or local environment"""
    if IN_COLAB:
        # In Colab, project should be in /content after cloning
        colab_path = Path('/content/Anomaly_Detect_LSTM_VAE')
        print(f"Checking Colab path: {colab_path}")
        print(f"Path exists: {colab_path.exists()}")
        if colab_path.exists():
            req_file = colab_path / 'requirements.txt'
            print(f"Requirements file exists: {req_file.exists()}")
            if req_file.exists():
                return colab_path
        print("Project directory not found in Colab")
        print("Current directory:", os.getcwd())
        print("Directory contents:", os.listdir())
        return None
    else:
        # Local development paths
        potential_locations = [
            Path('C:/Users/Augusto-PC/Documents/GitHub/LSTM'),
            Path(os.getcwd()),
            Path(os.path.expanduser('~/LSTM'))
        ]
        
        for location in potential_locations:
            try:
                if location.exists() and (location / 'requirements.txt').exists():
                    return location
            except Exception:
                continue
                
        print("Could not find project directory containing requirements.txt")
        return None

def setup_environment():
    """Setup the Python environment with required packages"""
    if IN_COLAB:
        print("Running in Google Colab environment")
        # Clone repository first if in Colab
        if not clone_repository():
            return False
    
    project_dir = get_project_dir()
    if project_dir is None:
        print("Failed to locate project directory")
        return False
        
    print(f"Found project directory at: {project_dir}")
    
    # Install requirements
    requirements_file = project_dir / 'requirements.txt'
    if requirements_file.exists():
        print("Installing requirements...")
        cmd = [sys.executable, "-m", "pip", "install", "-r", str(requirements_file)]
        result = run_command(cmd)
        if result is None:
            return False
    else:
        print(f"Requirements file not found at {requirements_file}")
        return False
    
    return True

def run_notebook(project_dir):
    """Run the main notebook"""
    notebook_path = project_dir / 'LSTM_VAE com ajustes.ipynb'
    if not notebook_path.exists():
        print(f"Notebook not found at {notebook_path}")
        return
        
    print(f"Running notebook: {notebook_path}")
    cmd = ["jupyter", "nbconvert", 
           "--to", "notebook", 
           "--execute",
           "--inplace",
           str(notebook_path)]
    run_command(cmd)

if __name__ == '__main__':
    if setup_environment():
        project_dir = get_project_dir()
        if project_dir:
            run_notebook(project_dir)